In [1]:
import pandas as pd 
import numpy as np
# from transformers import BertTokenizer
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
import torch
from torch.utils.data import Dataset, DataLoader


/Users/anirudhvalluru/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
df=pd.read_csv("/Users/anirudhvalluru/Desktop/LLM Project/ExtractedTweets.csv")
df_neutral = pd.read_csv("/Users/anirudhvalluru/Desktop/LLM Project/political_social_media.csv")

print(df.columns)
print(df_neutral.columns)

print(df_neutral['bias'])


Index(['Party', 'Handle', 'Tweet'], dtype='object')
Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'audience', 'audience:confidence', 'bias',
       'bias:confidence', 'message', 'message:confidence', 'orig__golden',
       'audience_gold', 'bias_gold', 'bioid', 'embed', 'id', 'label',
       'message_gold', 'source', 'text'],
      dtype='object')
0       partisan
1       partisan
2        neutral
3        neutral
4       partisan
          ...   
4995    partisan
4996    partisan
4997     neutral
4998     neutral
4999     neutral
Name: bias, Length: 5000, dtype: object


In [3]:
# df_bias= df_neutral['bias']

# df_bias['tweet'] = df_neutral['text']
# print(df_bias)
df_neutral = df_neutral.drop(columns = ['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'audience', 'audience:confidence',
       'bias:confidence', 'message', 'message:confidence', 'orig__golden',
       'audience_gold', 'bias_gold', 'bioid', 'embed', 'id', 'label',
       'message_gold', 'source'], axis=1)

print(df_neutral.columns)
df = df.drop(columns = ['Handle'],axis= 1)
df['bias'] = df['Party']
df = df.drop(columns = ['Party'],axis= 1)
df['text'] = df['Tweet']
df = df.drop(columns = ['Tweet'],axis= 1)
# print(df_neutral)
# df_neutral.drop(['bias'] = ['partisan'],axis =0)
df_neutral = df_neutral[~df_neutral['bias'].str.lower().isin(['partisan'])]

df_neutral = df_neutral.reset_index(drop=True)
print(df_neutral)

# print(df.columns)
dfc = pd.concat([df_neutral, df], axis=0, ignore_index=True)
dfc = dfc.reset_index(drop=True)

dfc = dfc.replace(to_replace='Democrat', value = 0)
dfc = dfc.replace(to_replace='neutral', value = 1)
dfc = dfc.replace(to_replace='Republican', value = 2)

print(dfc['bias'].unique())


Index(['bias', 'text'], dtype='object')
         bias                                               text
0     neutral  Please join me today in remembering our fallen...
1     neutral  RT @SenatorLeahy: 1st step toward Senate debat...
2     neutral  @BBCWorld, help us keep the kidnapped Nigerian...
3     neutral  Show your Arizona pride-choose your favorite S...
4     neutral  What a wonderful night at State Senator Ken Ya...
...       ...                                                ...
3684  neutral  Today I will be back in Bloomington for a ‰ÛÏM...
3685  neutral  I was honored to host the Discovery Channel in...
3686  neutral  (Taken from posted WOKV interview)   Congressm...
3687  neutral  Join me next week for a town hall in Ocala! I'...
3688  neutral  Foreign Affairs Committee Hearing on Syria. I ...

[3689 rows x 2 columns]
[1 0 2]


In [4]:
print(dfc)

       bias                                               text
0         1  Please join me today in remembering our fallen...
1         1  RT @SenatorLeahy: 1st step toward Senate debat...
2         1  @BBCWorld, help us keep the kidnapped Nigerian...
3         1  Show your Arizona pride-choose your favorite S...
4         1  What a wonderful night at State Senator Ken Ya...
...     ...                                                ...
90144     2  Check out my op-ed on need for End Executive O...
90145     2  Yesterday, Betty &amp; I had a great time lear...
90146     2  We are forever grateful for the service and sa...
90147     2  Happy first day of school @CobbSchools! #CobbB...
90148     2  #Zika fears realized in Florida. House GOP act...

[90149 rows x 2 columns]


In [5]:
# tokenizer = BertTokenizer.pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3) #3 for deomctratic, neutral, and republican

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from torch.utils.data import DataLoader, Dataset
import torch

class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            add_special_tokens=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [7]:
from sklearn.model_selection import train_test_split #since neutral is ~3000, maybe consider k-stratified fold

texts = dfc['text'].tolist()
labels = dfc['bias'].tolist() # use this for the targets categorcial 

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42) #80-20 split

train_dataset = TweetDataset(train_texts, train_labels, tokenizer, max_len=128)
val_dataset = TweetDataset(val_texts, val_labels, tokenizer, max_len=128)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [8]:
from transformers import AdamW
from torch.optim import lr_scheduler

optimizer = AdamW(model.parameters(), lr=.001, eps=1e-8) #adamw is good for this


scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1) 

loss_fn = torch.nn.CrossEntropyLoss() # good for multiclass


/Users/anirudhvalluru/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
from tqdm import tqdm


device = torch.device('cpu')
model.to(device)

def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0

    for batch in tqdm(data_loader): #usign progress bar
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(data_loader)

def eval_model(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total


In [ ]:
EPOCHS = 3

for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    train_loss = train_epoch(model, train_loader, loss_fn, optimizer, device)
    accuracy = eval_model(model, val_loader, device)

    print(f"Train Loss: {train_loss:.3f} | Validation Accuracy: {accuracy:.3f}")


Epoch 1/3


  2%|▏         | 83/4508 [06:38<2:52:38,  2.34s/it] 

In [ ]:
model.save_pretrained('./political_bias_model')
tokenizer.save_pretrained('./political_bias_model') # save model for later to use in other models


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification


model = BertForSequenceClassification.from_pretrained('./political_bias_model') # to load new model
tokenizer = BertTokenizer.from_pretrained('./political_bias_model')

def predict(text, model, tokenizer, device):
    model.eval()
    inputs = tokenizer.encode_plus(
        text,
        return_tensors="pt",
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()

    return prediction
